#### Using the 2018 American Community Survey Data to Feature Engineer:
#### SQL code for retrival can be seen here:

'''

`WITH acs_2018 AS (
  SELECT geo_id, total_pop, housing_units, aggregate_travel_time_to_work, gini_index,
    bachelors_degree_or_higher_25_64, median_income

  FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr`
),


acs_zipcode AS (
  SELECT *
  FROM acs_2018 a18
  JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` geo
  ON a18.geo_id = geo.zip_code
) 

SELECT geo_id, zip_code, total_pop, housing_units, aggregate_travel_time_to_work, gini_index,
  bachelors_degree_or_higher_25_64, median_income
FROM acs_zipcode `

'''

In [2]:
import numpy as np
import pandas as pd
acs = pd.read_csv('~/Desktop/NYCDSA/Capstone/data/census_feats.csv')

In [6]:
# total units for housing in zip code / total population
acs['per_person_housing'] = acs['housing_units'] / acs['total_pop'] 
# total commute time for people in zip / total population
acs['mean_commute_time'] = acs['aggregate_travel_time_to_work'] / acs['total_pop']
# number of people in the zip code with a bachelors or higher / total population
acs['bachelors_ormore'] = acs['bachelors_degree_or_higher_25_64'] / acs['total_pop']

In [8]:
acs = acs.drop('housing_units',axis=1).drop('aggregate_travel_time_to_work',axis=1).\
drop('bachelors_degree_or_higher_25_64',axis=1)

In [10]:
zillow = pd.read_csv('~/Desktop/NYCDSA/Capstone/data/zillow-long-multifamily.csv')

In [26]:
zillow

,RegionID,SizeRank,State,City,Metro,CountyName,zipcodes,date,rent
0,61639,1,NY,New York,New York-Newark-Jersey City,New York County,10025,2010-09,2930.00
1,84654,2,IL,Chicago,Chicago-Naperville-Elgin,Cook County,60657,2010-09,1447.00
2,61637,3,NY,New York,New York-Newark-Jersey City,New York County,10023,2010-09,2797.00
3,91982,4,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,77494,2010-09,NaN
4,84616,5,IL,Chicago,Chicago-Naperville-Elgin,Cook County,60614,2010-09,1437.00
...,...,...,...,...,...,...,...,...,...
210288,90755,1857,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,75202,2020-01,NaN
210289,94610,1858,UT,Ephraim,NaN,Sanpete County,84627,2020-01,1296.48
210290,84452,1859,IL,Oak Park,Chicago-Naperville-Elgin,Cook County,60301,2020-01,NaN
210291,94629,1860,UT,Mount Pleasant,NaN,Sanpete County,84647,2020-01,1221.00


In [44]:
df = zillow.merge(acs,how='left',left_on='zipcodes',right_on='zip_code')

In [45]:
nyc = df[df.City == 'New York']

In [46]:
# limit to one point in time just to assess features
nyc = nyc[nyc.date == '2018-01']
nyc = nyc.drop('Metro', axis =1).drop('City',axis=1).drop('State',axis=1)

In [49]:
nyc.CountyName = [x.replace(" County","") for x in nyc.CountyName]

In [51]:
from sklearn.linear_model import LinearRegression
ols = LinearRegression()